In [3]:
pip install kagglehub[pandas-datasets]

In [6]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

In [7]:
!kaggle datasets  download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 98% 1.04G/1.06G [00:07<00:00, 232MB/s]
100% 1.06G/1.06G [00:07<00:00, 158MB/s]


In [8]:
import zipfile

# Define the zip file path

# Unzipping the file
with zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')
